# Make Detections with Model

In [54]:
import pickle 
import csv
import os
import numpy as np
import mediapipe as mp # Import mediapipe
import cv2 # Import opencv


In [55]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [56]:
#initiating the model
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic # Mediapipe Solutions
mp_pose = mp.solutions.pose

cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
                                 
        
        # 2. Right hand
        #mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
         #                        mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
          #                       mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
           #                      )

        # Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


In [57]:
results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST]

x: 0.28016942739486694
y: 0.8531628251075745
z: -2.433043956756592
visibility: 0.9110228419303894

In [58]:
num_coords = len(results.pose_landmarks.landmark)
num_coords

33

In [59]:
with open(r'C:\Users\hagar\OneDrive - mail.tau.ac.il\Desktop\Stage\LPC_2022\Hand decoder\Position\code\distance_position.pkl', 'rb') as f:
    model = pickle.load(f)

In [60]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [112]:
landmarks = ['frame', 'time','predicted_class','prob_selected_class ','prob_all_classes','prob_0','prob_1','prob_2','prob_3','prob_4','d_x','d_y','d_z']
# for val in range(1, num_coords+1):
    # landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [113]:
fn = 'Cinema' #name of the video we want to know its positions

In [115]:
file_path = f'C:/Users/hagar/OneDrive - mail.tau.ac.il/Desktop/Stage/LPC_2022/Hand decoder/Position/results/predicted_csv/{fn}_pose_estimation_by_frames.csv'

with open(file_path,mode='w', newline='') as f: 
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [116]:
cap = cv2.VideoCapture(f"C:/Users/hagar/OneDrive - mail.tau.ac.il/Desktop/Stage/LPC_2022/Hand decoder/Position/data/test_videos/{fn}.mp4")
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

size = (frame_width, frame_height)


marked_video = cv2.VideoWriter(f'C:/Users/hagar/OneDrive - mail.tau.ac.il/Desktop/Stage/LPC_2022/Hand decoder/Position/results/marked_videos/{fn}_marked.avi',cv2.VideoWriter_fourcc(*'MJPG'),30, size)

fps = cap.get(cv2.CAP_PROP_FPS) # get the frmae number

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    n=0
    while cap.isOpened():
        n = n+1
        ret, frame = cap.read()
        if not ret:
            break
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(image)
        
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
       # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(190,57,255), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(57,255,156), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(255,20,217), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )
        # Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=1, circle_radius=2),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=1, circle_radius=2)
                                 )
                        

        # Export coordinates
        try:
            # Extract Pose landmarks
            # relevant_landmarks = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST], results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE]
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            row = pose_row

            
            # Name colls
            num_coords = len(pose)
            col_name = []
            for val in range(1, num_coords+1):
                col_name += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val),'v{}'.format(val)]
            
            
            # Make Detections 
            X = pd.DataFrame([row], columns = col_name)

            # Create distance colls
            X['d_x'] = X['x1'] - X['x17']
            X['d_y'] = X['y1'] - X['y17']
            X['d_z'] = X['z1'] - X['z17']
            
            relevant_cols = ['d_x','d_y','d_z']
            X = X.filter(relevant_cols, axis=1)
            

            predicted_position = model.predict(X)[0] # [0] because predict returns 2 arg and we want only the first one
            position_prob = model.predict_proba(X)[0]
            prob_to_display = str(round(position_prob[np.argmax(position_prob)],2))
            prob_0 = round(position_prob[0],2)
            prob_1 = round(position_prob[1],2)
            prob_2 = round(position_prob[2],2)
            prob_3 = round(position_prob[3],2)
            prob_4 = round(position_prob[4],2)
            
            # Append frame, prediction class and probability 
            row.insert(0, n)
            row.insert(1, round(n/fps,3))
            row.insert(2, predicted_position)
            row.insert(3, prob_to_display)
            row.insert(4, position_prob)
            row.insert(5, prob_0)
            row.insert(6, prob_1)
            row.insert(7, prob_2)
            row.insert(8, prob_3)
            row.insert(9, prob_4)
            


            
            # Write prediction to a CSV
            with open(file_path, mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)
            
                
            # Write prediction on the video
            font = cv2.FONT_HERSHEY_SIMPLEX
            
            # Get status box on the top left corner
            # cv2.rectangle(image, (0,0), (250*2, 60*2), (245, 117, 16), -1)
            cv2.rectangle(image, (920,400), (1500, 520), (138,24,119), -1)

            
            # Display Class
            cv2.putText(image, 'Predicted Position'
                         , (940,430), font, 1, (255, 255, 255), 1, cv2.LINE_AA)
            cv2.putText(image, str(predicted_position)
                         , (940,500), font, 2, (255, 255, 255), 2, cv2.LINE_AA)
                    
            
            # Display Probability
            cv2.putText(image, 'Probability'
                        , (1300,430), font, 1, (255, 255, 255), 1, cv2.LINE_AA)

            
            cv2.putText(image, prob_to_display
                        , (1300,500), font, 2, (255, 255, 255), 2, cv2.LINE_AA)
            
            
            
        except Exception as e:
                print(e)
                pass
            


        cv2.imshow('cued_estimated', image)
        marked_video.write(image)


        if cv2.waitKey(10) & 0xFF == ord('q'):
            break


marked_video.release()
cap.release()
cv2.destroyAllWindows()
print("The video was successfully saved")

The video was successfully saved


In [110]:
position_prob[4]

0.98